In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import seisbench.data as sbd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight

/home/namithk/miniconda3/envs/eew/lib/python3.9/site-packages/seisbench/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
dataset = sbd.STEAD()

label_map = {"noise": 0, "earthquake_local": 1}
dataset.metadata["label"] = dataset.metadata["trace_category"].map(label_map)

train_indices = dataset.metadata[dataset.metadata["split"] == "train"].index
dev_indices = dataset.metadata[dataset.metadata["split"] == "dev"].index

# 🔥 SANITY MODE (20% training data)
train_indices = train_indices[:int(0.2 * len(train_indices))]


2026-03-01 10:51:11,014 | seisbench | WARNING | Output component order not specified, defaulting to 'ZNE'.


In [4]:
class SteadTorchDataset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        waveform = self.dataset.get_waveforms([real_idx])[0]

        # Per-trace normalization
        waveform = waveform - waveform.mean(axis=1, keepdims=True)
        waveform = waveform / (waveform.std(axis=1, keepdims=True) + 1e-6)

        label = dataset.metadata.iloc[real_idx]["label"]

        return torch.tensor(waveform, dtype=torch.float32), \
               torch.tensor(label, dtype=torch.float32)

train_dataset = SteadTorchDataset(dataset, train_indices)
dev_dataset = SteadTorchDataset(dataset, dev_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, num_workers=0)

In [5]:
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size,
                              padding=0, dilation=dilation)

    def forward(self, x):
        x = F.pad(x, (self.padding, 0))
        return self.conv(x)


In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, channels, kernel_size, dilation):
        super().__init__()
        self.conv1 = CausalConv1d(channels, channels, kernel_size, dilation)
        self.bn1 = nn.BatchNorm1d(channels)
        self.conv2 = CausalConv1d(channels, channels, kernel_size, dilation)
        self.bn2 = nn.BatchNorm1d(channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(self.bn2(self.conv2(out)))
        out += residual
        return self.relu(out)

In [7]:
class ResidualCausalTCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_conv = nn.Conv1d(3, 64, kernel_size=1)

        self.blocks = nn.Sequential(
            ResidualBlock(64, 5, 1),
            ResidualBlock(64, 5, 2),
            ResidualBlock(64, 5, 4),
            ResidualBlock(64, 5, 8),
            ResidualBlock(64, 5, 16),
            ResidualBlock(64, 5, 32),
            ResidualBlock(64, 5, 64),
            ResidualBlock(64, 5, 128),
            ResidualBlock(64, 5, 256)
        )

        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = self.input_conv(x)
        x = self.blocks(x)
        x = self.pool(x).squeeze(-1)
        return self.fc(x)

model = ResidualCausalTCN().to(device)

In [8]:
labels = dataset.metadata.loc[train_indices]["label"].values
weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
pos_weight = torch.tensor([weights[0] / weights[1]]).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [9]:
def train_one_epoch():
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate():
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for x, y in dev_loader:
            x, y = x.to(device), y.unsqueeze(1).to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == y).sum().item()
            total += y.size(0)
    return total_loss / len(dev_loader), correct / total

In [10]:
num_epochs = 2
train_losses, val_losses, val_accuracies = [], [], []

for epoch in range(num_epochs):
    train_loss = train_one_epoch()
    val_loss, val_acc = evaluate()

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val Accuracy: {val_acc:.4f}")
    print("-" * 30)

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train_losses, marker='o', label="Training Loss")
plt.plot(val_losses, marker='s', label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("causal_tcn_loss.png", dpi=300)
plt.show()

plt.figure(figsize=(8,5))
plt.plot(val_accuracies, marker='o')
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy")
plt.grid(True)
plt.tight_layout()
plt.savefig("causal_tcn_accuracy.png", dpi=300)
plt.show()